In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.5 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 3.3 MB 32.0 MB/s 
     |████████████████████████████████| 636 kB 44.1 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import torch
print(torch.cuda.is_available())

SyntaxError: ignored

In [2]:
!pip list

Package                       Version
----------------------------- --------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   21.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4                           0.0.1
CacheControl

In [ ]:
import os
class Config:
    def __init__(self):
        self.train_data_path=r"/content/char_ner_train.csv"
        self.saved_ids_data_path=r"/content/saved_ids.pkl"
        self.model_save_path='/content/bert-softmax.bin'

        self.label2id={
            'M-GPE':0,
            'M-ORG':1,
            'M-PER':2,
            'M-LOC':3,
            'S-ORG':4,
            'E-ORG':5,
            'S-PER':6,
            'E-LOC':7,
            'B-LOC':8,
            'B-ORG':9,
            'S-GPE':10,
            'S-LOC':11,
            'B-PER':12,
            'E-GPE':13,
            'O':14,
            'B-GPE':15,
            'E-PER':16,
        }
        self.num_labels=17
        self.hidden_dropout_prob= 0.1
        self.hidden_size=768
        self.epoch_num=20
        self.batch_size=25
        self.learning_rate=3e-5
        # 训练集、验证集划分比例
        self.train_test_split_size = 0.005
        self.device = torch.device("cuda")
config=Config()
print(os.getcwd())

/content


### data_set,数据集构建

In [ ]:
import pickle
from transformers import BertModel, BertTokenizer
from torch.utils.data import Dataset
import torch
from torch.nn.utils.rnn import pad_sequence

class MyDataSet(Dataset):

    def __init__(self,wordlists,taglists):
        self.wordlists = wordlists
        self.taglists =taglists

    def __getitem__(self, item):
        return torch.Tensor(self.wordlists[item]), torch.Tensor(self.taglists[item])

    def __len__(self):
        return len(self.wordlists)


def collate_fn(batch):
    """
    :param batch: (batch_num, ([sentence_len, word_embedding], [sentence_len]))
    :return:
    """
    a,b=len(batch),len(batch[0])
    x_list = [x[0] for x in batch]
    y_list = [x[1] for x in batch]
    lengths = [len(item[0]) for item in batch]
    x_list = pad_sequence(x_list, padding_value=0)
    y_list = pad_sequence(y_list, padding_value=-1)

    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

    #return x_list.transpose(0, 1).to(config.device), y_list.transpose(0, 1).to(config.device), lengths.to(config.device)
    return x_list.transpose(0, 1), y_list.transpose(0, 1), lengths

# Model,最基本的模型文件

In [ ]:
from transformers import BertModel, BertTokenizer
from torch.nn import Module
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torch

class BertNER(Module):
    def __init__(self):
        super(BertNER, self).__init__()
        self.num_labels = config.num_labels

        self.bert = BertModel.from_pretrained('bert-base-chinese', output_hidden_states=True)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        #self.init_weights()

    def forward(self, input_data, lengths ):

        # 求出attention_mask
        max_len = int(max(lengths))
        attention_mask = torch.Tensor()
        # length = length.numpy()
        # 与每个序列等长度的全1向量连接长度为最大序列长度-当前序列长度的全0向量。
        for len_ in lengths:
            attention_mask = torch.cat((attention_mask, torch.Tensor([[1] * len_ + [0] * (max_len - len_)])), dim=0)


        input_data=input_data.long()
        attention_mask=attention_mask.long()

        outputs = self.bert(input_data.to(config.device),
                            attention_mask=attention_mask.to(config.device),)
        sequence_output = outputs[0]
        #print("sequence_output,shape:",sequence_output.shape)

        origin_sequence_output=[]
        # 去掉special token, 待改进
        for idx, sentence in enumerate(sequence_output):
            #print(len(sentence),lengths[idx])
            sentence=torch.index_select(sentence,0,torch.arange(1,lengths[idx]-1).to(config.device))
            origin_sequence_output.append(sentence)

        #print('origin_sequence_output:',len(origin_sequence_output),len(origin_sequence_output[0]),len(origin_sequence_output[1]))
        # 将sequence_output的pred_label维度padding到最大长度
        padded_sequence_output = pad_sequence(origin_sequence_output, batch_first=True)
        # dropout pred_label的一部分feature
        padded_sequence_output = self.dropout(padded_sequence_output)
        # 得到判别值
        # (batch_size,seq,label_num)
        logits = self.classifier(padded_sequence_output)

        return logits

    def get_loss(self,output_logits,labels):

        labels=labels.long().to(config.device)
        #output_logits=output_logits.long()
        if labels is not None:
            loss_mask = labels.gt(-1)
            loss_fct = CrossEntropyLoss()
            # Only keep active parts of the loss
            if loss_mask is not None:
                # 只留下label存在的位置计算loss
                # (batch_size*seq,)
                active_loss = loss_mask.reshape(-1) == 1
                #print('active_loss',active_loss.shape)
                # (batch_size*seq, label_nums)
                active_logits = output_logits.reshape(-1, self.num_labels)#
                #print('active_logits',active_logits.shape)
                active_logits = active_logits[active_loss]
                # (batch_size*seq,)
                active_labels = labels.reshape(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(output_logits.reshape(-1, self.num_labels), labels.reshape(-1))
        #print('loss',loss.shape)
        return loss



# run_and_train

### evaluate,计算f1值的

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
import torch

def get_f1(model, test_dataLoader):
    with torch.no_grad():

        all_y_pred=[]
        all_y_true=[]
        for idx, batch_data in enumerate(test_dataLoader):
            x, labels, lengths = batch_data
            logits = model.forward(x,lengths).cpu()

            # 最大值简单解码
            decoded_outputs = np.argmax(logits, axis=2)

            # 思路是先展平,然后根据loss_mask选出非PADDING值,最后送入f1计算
            # loss_mask,概念和get_loss里的那个变量一样
            loss_mask = labels.gt(-1).reshape(-1)

            # 将输出和标签展开
            y_pred = decoded_outputs.reshape(-1)[loss_mask]
            y_true = labels.reshape(-1)[loss_mask]

            all_y_pred.append(y_pred)
            all_y_true.append(y_true)

        all_y_true=np.concatenate(all_y_true,axis=0)
        all_y_pred = np.concatenate(all_y_pred, axis=0)

        f1_macro = f1_score(all_y_true, all_y_pred, average='macro')
        f1_micro = f1_score(all_y_true, all_y_pred, average='micro')

        return f1_macro, f1_micro

### 读取,保存 simple_train,简单训练

In [ ]:
from torch.optim import AdamW
from torch.utils.data import DataLoader
import logging
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import nn
from torch.nn import init
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 先加载分词好的数据
file = open(config.saved_ids_data_path, "rb")
word_lists,tag_lists=pickle.load(file)
file.close()
logging.info("--------已加载好训练数据!--------")

# 分割训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(word_lists[:7000], tag_lists[:7000], test_size=config.train_test_split_size, random_state=0)

# 构建dataloader
train_dataset=MyDataSet(x_train,y_train)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, collate_fn=collate_fn)
test_dataset=MyDataSet(x_test,y_test)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=config.batch_size, collate_fn=collate_fn)
logging.info("--------已构建好DataLoader!--------")

# 这里加载模型
model=BertNER()
model=torch.load(config.model_save_path)
model.to(config.device)
logging.info("--------已准备好模型!--------")

# 构建优化器
param_optimizer = list(model.classifier.named_parameters())
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=config.learning_rate)#, correct_bias=False)
logging.info("--------已构建优化器!--------")

# 计算当前模型的F1值,用于等会做对比
best_f1=get_f1(model,test_dataloader)[0]

# 开始训练
for epoch in range(1,config.epoch_num+1):
    for batch_num,batch_data in enumerate(tqdm(train_dataloader)):
        #print(batch_num,'begin')
        x,y,lengths =batch_data
        output_logits=model(x,lengths)
        loss=model.get_loss(output_logits ,y)
        model.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num%50==0:
          current_f1=get_f1(model,test_dataloader)[0]
          print("loss:", loss.item(),"  f1_score:",current_f1)
          if current_f1-best_f1>0.01:
            model_save_path='/content/bert-softmax.bin'
            torch.save(model,config.model_save_path)
            best_f1=current_f1
            print("已经保存最好模型")
           # print("f1_score:",get_f1(model,test_dataloader))


        #print(batch_num,loss)



Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/279 [00:00<?, ?it/s]

loss: 0.12149973213672638   f1_score: 0.6169880408477023


  0%|          | 1/279 [00:04<20:56,  4.52s/it]

已经保存最好模型


 18%|█▊        | 50/279 [01:13<06:03,  1.59s/it]

...

 90%|████████▉ | 251/279 [06:06<00:56,  2.02s/it]

loss: 0.06856632232666016   f1_score: 0.7179615570312319


  0%|          | 1/279 [00:03<16:13,  3.50s/it]

loss: 0.12283065915107727   f1_score: 0.6911587627305785


  3%|▎         | 8/279 [00:12<06:11,  1.37s/it]